<a href="https://colab.research.google.com/github/inathanms/Evolucao_Software_2025-2_goose/blob/main/Evolucao_Software_2025_2_goose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Fetch the last 100 closed pull request comments from the "block/goose" GitHub repository using the GitHub API, excluding bot comments. Map the following acronyms in the comments to their full forms: WIP -> Work In Progress, LGTM -> Looks Good To Me, PR -> Pull Request, MR -> Merge Request, IMO -> In My Opinion, IMHO -> In My Humble Opinion, AFAIK -> As Far As I Know, FYI -> For Your Information, Nit -> Nitpick, TL;DR -> Too Long; Didn't Read, TBD -> To Be Defined / To Be Determined, ACK -> Acknowledge, PTAL -> Please Take Another Look. Display the resulting list of comments.

## Set up github api access

### Subtask:
Obtain a GitHub personal access token with the necessary permissions to read repository data.


## Fetch closed pull requests

### Subtask:
Use the GitHub API to retrieve the last 100 closed pull requests for the `block/goose` repository.


**Reasoning**:
Import the necessary library and set up the API request to fetch the last 100 closed pull requests.



In [36]:
import requests
from google.colab import userdata

# Define the repository details
repo_owner = "block"
repo_name = "goose"

# GitHub API endpoint for pull requests
api_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/pulls"

# Get GitHub token from environment variable
github_token = userdata.get("GITHUB_ACCESS_TOKEN")

# Set up the request parameters
params = {
    "state": "closed",
    "per_page": 100,
    "sort": "created",
    "direction": "desc"
}

# Set up the request headers with authentication
headers = {
    "Authorization": f"token {github_token}",
    "Accept": "application/vnd.github.v3+json"
}

# Make the GET request to the GitHub API
response = requests.get(api_url, params=params, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    closed_pull_requests = response.json()
    print(f"Successfully retrieved {len(closed_pull_requests)} closed pull requests.")
else:
    print(f"Error: {response.status_code} - {response.text}")
    closed_pull_requests = []
closed_pull_requests

Successfully retrieved 100 closed pull requests.


[{'url': 'https://api.github.com/repos/block/goose/pulls/5382',
  'id': 2948367935,
  'node_id': 'PR_kwDOMneZ986vvIY_',
  'html_url': 'https://github.com/block/goose/pull/5382',
  'diff_url': 'https://github.com/block/goose/pull/5382.diff',
  'patch_url': 'https://github.com/block/goose/pull/5382.patch',
  'issue_url': 'https://api.github.com/repos/block/goose/issues/5382',
  'number': 5382,
  'state': 'closed',
  'locked': False,
  'title': 'fixing typo in blog metadata',
  'user': {'login': 'blackgirlbytes',
   'id': 22990146,
   'node_id': 'MDQ6VXNlcjIyOTkwMTQ2',
   'avatar_url': 'https://avatars.githubusercontent.com/u/22990146?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/blackgirlbytes',
   'html_url': 'https://github.com/blackgirlbytes',
   'followers_url': 'https://api.github.com/users/blackgirlbytes/followers',
   'following_url': 'https://api.github.com/users/blackgirlbytes/following{/other_user}',
   'gists_url': 'https://api.github.com/users/blackgirlb

## Fetch comments for each pr

### Subtask:
For each retrieved pull request, fetch its comments using the GitHub API.


**Reasoning**:
Iterate through the retrieved pull requests and fetch comments for each using the provided comments URL in the pull request object.



In [38]:
all_comments = []

if closed_pull_requests:
    for pr in closed_pull_requests:
        # Fetch regular comments
        comments_url = pr["_links"]["comments"]["href"]
        comments_response = requests.get(comments_url, headers=headers)

        if comments_response.status_code == 200:
            all_comments.extend(comments_response.json())
        else:
            print(f"Error fetching comments for PR {pr['number']}: {comments_response.status_code} - {comments_response.text}")

        # Fetch review comments
        review_comments_url = pr["_links"]["review_comments"]["href"]
        review_comments_response = requests.get(review_comments_url, headers=headers)

        if review_comments_response.status_code == 200:
            all_comments.extend(review_comments_response.json())
        else:
            print(f"Error fetching review comments for PR {pr['number']}: {review_comments_response.status_code} - {review_comments_response.text}")
else:
    print("No closed pull requests were retrieved in the previous step.")

## Filter out bot comments

### Subtask:
Identify and remove comments made by bots.

**Reasoning**:
Iterate through the fetched comments and exclude those where the `type` of the author is 'Bot'.

In [39]:
human_comments = [comment for comment in all_comments if comment['user']['type'] != 'Bot']

print(f"Filtered out {len(all_comments) - len(human_comments)} bot comments.")

[{'url': 'https://api.github.com/repos/block/goose/issues/comments/3448433215', 'html_url': 'https://github.com/block/goose/pull/5382#issuecomment-3448433215', 'issue_url': 'https://api.github.com/repos/block/goose/issues/5382', 'id': 3448433215, 'node_id': 'IC_kwDOMneZ987Niuo_', 'user': {'login': 'github-actions[bot]', 'id': 41898282, 'node_id': 'MDM6Qm90NDE4OTgyODI=', 'avatar_url': 'https://avatars.githubusercontent.com/in/15368?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/github-actions%5Bbot%5D', 'html_url': 'https://github.com/apps/github-actions', 'followers_url': 'https://api.github.com/users/github-actions%5Bbot%5D/followers', 'following_url': 'https://api.github.com/users/github-actions%5Bbot%5D/following{/other_user}', 'gists_url': 'https://api.github.com/users/github-actions%5Bbot%5D/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/github-actions%5Bbot%5D/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/github-actions

## Extract and store comments

### Subtask:
Collect the text of the remaining comments into a list.

**Reasoning**:
Iterate through the filtered comments and extract the `body` (the comment text) from each comment object.

In [40]:
comment_texts = [comment['body'] for comment in human_comments]

print(f"Extracted {len(comment_texts)} human comments.")

Extracted 317 human comments.
["Thanks for the contribution! The blog post is well written. \r\n\r\n- Can you fix the failing DCO check -- here's how https://github.com/block/goose/pull/5381/checks?check_run_id=53686569106\r\n- Ensure all mentions of goose use a lower case goose instead?\r\n\r\n\r\nAfter that I can approve and merge", 'Done @blackgirlbytes ', 'thanks the DCO check is passing now, but I dont see that you did the second part..in your blog post..can you just quickly change mentions of Goose that are upper case to use lowercase goose..for example where you say "Goose will prompt the user for it once via the CLI" instead say "goose will prompt the user for it once via the CLI"', 'Hi @blackgirlbytes , thank you for pointing out. Now i have changed every mention of goose into lowercase except for the variable names used', 'awesome -- merging!', '```suggestion\n<iframe class="aspect-ratio" src="https://www.youtube.com/embed/-_1GALH2ER0" title="Vibe Code with goose: Automate my

## Map acronyms

### Subtask:
Replace the specified acronyms in the collected comments with their full forms.

**Reasoning**:
Iterate through the extracted comment texts and use string replacement to substitute the acronyms with their corresponding full forms.

In [41]:
acronym_map = {
    "WIP": "Work In Progress",
    "LGTM": "Looks Good To Me",
    "PR": "Pull Request",
    "MR": "Merge Request",
    "IMO": "In My Opinion",
    "IMHO": "In My Humble Opinion",
    "AFAIK": "As Far As I Know",
    "FYI": "For Your Information",
    "Nit": "Nitpick",
    "TL;DR": "Too Long; Didn't Read",
    "TBD": "To Be Defined / To Be Determined",
    "ACK": "Acknowledge",
    "PTAL": "Please Take Another Look"
}

mapped_comments = []
for comment in comment_texts:
    mapped_comment = comment
    for acronym, full_form in acronym_map.items():
        # Use regex to replace whole words only to avoid replacing parts of other words
        import re
        mapped_comment = re.sub(r'\b' + re.escape(acronym) + r'\b', full_form, mapped_comment, flags=re.IGNORECASE)
    mapped_comments.append(mapped_comment)

print("Mapped acronyms in comments.")

Mapped acronyms in comments.


## Display results

### Subtask:
Show the processed list of comments.

**Reasoning**:
Display the `mapped_comments` list to show the final result.

In [42]:
for comment in mapped_comments:
    print(comment)

Thanks for the contribution! The blog post is well written. 

- Can you fix the failing DCO check -- here's how https://github.com/block/goose/pull/5381/checks?check_run_id=53686569106
- Ensure all mentions of goose use a lower case goose instead?


After that I can approve and merge
Done @blackgirlbytes 
thanks the DCO check is passing now, but I dont see that you did the second part..in your blog post..can you just quickly change mentions of Goose that are upper case to use lowercase goose..for example where you say "Goose will prompt the user for it once via the CLI" instead say "goose will prompt the user for it once via the CLI"
Hi @blackgirlbytes , thank you for pointing out. Now i have changed every mention of goose into lowercase except for the variable names used
awesome -- merging!
```suggestion
<iframe class="aspect-ratio" src="https://www.youtube.com/embed/-_1GALH2ER0" title="Vibe Code with goose: Automate my workflow" frameborder="0" allow="accelerometer; autoplay; clipboa